In [1]:
import numpy as np
import pandas as pd
import glob
import re

In [2]:
datadir = "../data/McGill-Billboard/"
# filepaths = glob.glob(datadir+'[0-9][0-9][0-9][0-9]/salami_chords.txt')
filepaths = glob.glob(datadir+'1256/salami_chords.txt')

In [3]:
fileidx = 0
filepath = filepaths[fileidx]

def ExtractFile(filepath):
    f = open(filepath, 'r')
    raw = f.readlines()
    f.close
    return raw

def ExtractMeta(raw):
    title = re.split('# title: |\n', raw[0])[1]
    artist = re.split('# artist: |\n', raw[1])[1]
    metre = re.split('# metre: |\n', raw[2])[1]
    tonic = re.split('# tonic: |\n', raw[3])[1]

    return (title, artist, metre, tonic)

raw = ExtractFile(filepath)
(title, artist, metre, tonic) = ExtractMeta(raw)

In [4]:
nonmeta = raw[6:-2]

def ExtractPhrases(raw):
    raw = raw[6:-2]
    phrases = list()
    for row in raw:
        splitted = re.split('\t', row)[1:][0]
        splitted = re.split('\|', splitted)[1:]
        
        if not splitted: continue
        
        phrase = splitted[:-1]

        # check for repeat
        if IsRepeat(splitted[-1][1:3]):
            phrase.append(splitted[-1][1:3])

        phrases.append(phrase)
    return phrases

def CleanPhrases(phrases):
    phrases_clean = list()
    for phrase in phrases:
        phrase_clean = [chord.strip() for chord in phrase]
        phrases_clean.append(phrase_clean)
    return phrases_clean
    
def FormatChords(phrases_clean):
    chords = list()
    
    for phrase in phrases_clean:
        # clean non-chords
        phrase_chord = [RemoveNoise(chord) for chord in phrase]
        
        # remove blank (noise-only) lines
        if all([not p for p in phrase_chord]): continue

        # check for repeat
        if IsRepeat(phrase_chord[-1]):
            repeated = phrase_chord[:-1]
            n_repeat = int(phrase_chord[-1][-1])
            phrase_chord = repeated*n_repeat

        chords.append(phrase_chord)
    # flatten 2d list
    chords = sum(chords, [])

    # split chords into bars
    chords = [chord.split() for chord in chords]

    return chords

def IsRepeat(str):
    if re.match(r'x\d', str):
        return True
    else:
        return False

def RemoveNoise(chord):
    chord_clean = re.sub(r'\.|\*|\(\d+/\d+\)|N', '', chord)
    chord_clean = chord_clean.split()
    chord_clean = ' '.join(chord_clean)
    return chord_clean

phrases = ExtractPhrases(raw)
phrases_clean = CleanPhrases(phrases)
chords = FormatChords(phrases_clean)

In [5]:
def CreateIndex(chords):
    indexs_bar = list()
    for i, bar in enumerate(chords):
        index_bar = [i]*len(bar)
        indexs_bar.append(index_bar)
    return indexs_bar
chords_index = CreateIndex(chords)

In [25]:
def Raw2Basic(chord_raw):
    root = chord_raw.split(':')[0]
    func = chord_raw.split(':')[1].split('/')[0]
    
    if func[:3]=='maj':
        func = 'maj'
    elif func[:3]=='min':
        func = 'min'
    elif func[:3]=='sus':
        func = func[:4]
    elif func[0]=='1' or func[0]=='5':
        func = '15'
    
    chord_basic = root+':'+func
    return chord_basic

def Raw2Root(chord_raw):
    root = chord_raw.split(':')[0]
    return Raw2Root

In [6]:
df_chords = pd.DataFrame()
df_chords['bar'] = sum(chords_index, [])
df_chords['chords'] = sum(chords, [])
df_chords['chords_basic'] = df_chords['chords'].apply(Raw2Basic)
df_chords['chords_root'] = df_chords['chords'].apply(Raw2Root)
df_chords.iloc[10:30]

,bar,chords
10,10,C:min7
11,11,C:min
12,12,Bb:5/9
13,13,Bb:5/9
14,13,C:5
15,14,Bb:5/9
16,15,Bb:5/9
17,15,C:5
18,16,C:maj
19,17,C:maj


In [26]:
df_chords['chords'].apply(Raw2Basic)

0        C:15
1        C:15
2        C:15
3        C:15
4       C:min
        ...  
302    G:sus4
303    G:sus4
304    G:sus4
305    Bb:maj
306     C:maj
Name: chords, Length: 307, dtype: object